In [ ]:
import numpy as np
import pandas as pd
import os
from scipy import optimize

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    adjust_yaxis(ax2,(y1-y2)/2,v2)
    adjust_yaxis(ax1,(y2-y1)/2,v1)

def adjust_yaxis(ax,ydif,v):
    """shift axis ax by ydiff, maintaining point v at the same location"""
    inv = ax.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, ydif))
    miny, maxy = ax.get_ylim()
    miny, maxy = miny - v, maxy - v
    if -miny>maxy or (-miny==maxy and dy > 0):
        nminy = miny
        nmaxy = miny*(maxy+dy)/(miny+dy)
    else:
        nmaxy = maxy
        nminy = maxy*(miny+dy)/(maxy+dy)
    ax.set_ylim(nminy+v, nmaxy+v)
    
def fit(x, x0, x1, b, k1, k2, k3):
    condlist = [x < x0, (x >= x0) & (x < x1), x >= x1]
    funclist = [lambda x: k1*x + b, lambda x: k1*x0 + b + k2*(x-x0), lambda x: k1*x0 + b + k2*(x1-x0) + k3*(x - x1)]
    return np.piecewise(x, condlist, funclist)

def invfit(x, x0, x1, b, k1, k2, k3):
    condlist = [x < k1*x0+b, (x >= k1*x0+b) & (x < k1*x0 + b + k2*(x1-x0)), x >= k1*x0 + b + k2*(x1-x0)]
    funclist = [lambda x: (x - b) / k1, lambda x: (x - k1*x0 - b) / k2 + x0, lambda x: (x - k1*x0 - b - k2*(x1-x0)) / k3 + x1]
    return np.piecewise(x, condlist, funclist)

def getinvcoeffs(x0, x1, b, k1, k2, k3):
    return np.array([k1*x0+b, k1*x0 + b + k2*(x1-x0), -b / k1, 1.0/k1, 1.0/k2, 1.0/k3])

In [ ]:
esc_file=os.path.expanduser('~/Documents/log/2019-01-17T09-25-04/logstream_ActAdapEsc.bin')
torque_thrust_file=os.path.expanduser('~/Documents/Log_2019-01-17_092649.csv')

In [ ]:
esc_fmt = np.dtype([('idx', '>u1'),
               ('cmdSec', '>u4'), ('cmdUsec', '>u4'), ('cmd', '>u4'), ('cmdVel', '>f8'),
               ('fbSec', '>u4'), ('fbUsec', '>u4'), ('angVel', '>f8'),
               
               ('length', '>u2'),
               ('counts', '>u2'),
               ('voltage', '>u2'),
               ('temperature', '>u2'),
               ('current', '>u2'),
               ('id', 'u1')])

In [ ]:
esc_dat = np.fromfile(esc_file, dtype=esc_fmt)
esc = pd.DataFrame(esc_dat.byteswap().newbyteorder())

torque_thrust = pd.read_csv(torque_thrust_file)
torque=torque_thrust.loc[(~torque_thrust['Time (s)'].isna()) & (~torque_thrust['Torque (N\xc2\xb7m)'].isna())]
thrust=torque_thrust.loc[(~torque_thrust['Time (s)'].isna()) & (~torque_thrust['Thrust (kgf)'].isna())]

mot0=esc.loc[(esc['idx'] == 0) & (esc['cmd'] <= 800)]
mot1=esc.loc[(esc['idx'] == 1) & (esc['cmd'] <= 800)]
mot2=esc.loc[(esc['idx'] == 2) & (esc['cmd'] <= 800)]
mot3=esc.loc[(esc['idx'] == 3) & (esc['cmd'] <= 800)]

mot0_cmdTime = mot0.cmdSec + mot0.cmdUsec / (1000.0 * 1000.0)
mot1_cmdTime = mot1.cmdSec + mot1.cmdUsec / (1000.0 * 1000.0)
mot2_cmdTime = mot2.cmdSec + mot2.cmdUsec / (1000.0 * 1000.0)
mot3_cmdTime = mot3.cmdSec + mot3.cmdUsec / (1000.0 * 1000.0)

mot0_fbTime = mot0.fbSec + mot0.fbUsec / (1000.0 * 1000.0)
mot1_fbTime = mot1.fbSec + mot1.fbUsec / (1000.0 * 1000.0)
mot2_fbTime = mot2.fbSec + mot2.fbUsec / (1000.0 * 1000.0)
mot3_fbTime = mot3.fbSec + mot3.fbUsec / (1000.0 * 1000.0)

In [ ]:
cmdHi = mot0.cmd[(mot0.cmd > 400) & (mot0.angVel > 2000)].values.astype("float64")
angVelHi = mot0.angVel[(mot0.cmd > 400) & (mot0.angVel > 2000)].values
A = np.vstack([cmdHi, np.ones(len(cmdHi))]).T
mHi, cHi = np.linalg.lstsq(A, angVelHi)[0]
print mHi, cHi
plt.plot(cmdHi, angVelHi, '*')
plt.plot(cmdHi, mHi*cmdHi + cHi)

In [ ]:
cmdMed = mot0.cmd[(mot0.cmd > 100) & (mot0.angVel > 1000)].values.astype("float64")
angVelMed = mot0.angVel[(mot0.cmd > 100) & (mot0.angVel > 1000)].values
A = np.vstack([cmdMed, np.ones(len(cmdMed))]).T
mMed, cMed = np.linalg.lstsq(A, angVelMed)[0]
print mMed, cMed
plt.plot(cmdMed, angVelMed, '*')
plt.plot(cmdMed, mMed*cmdMed + cMed)

In [ ]:
cmdLow = mot0.cmd[(mot0.cmd > 0) & (mot0.cmd < 200) & (mot0.angVel < 1200)].values.astype("float64")
angVelLow = mot0.angVel[(mot0.cmd > 0) & (mot0.cmd < 200) & (mot0.angVel < 1200)].values
A = np.vstack([cmdLow, np.ones(len(cmdLow))]).T
mLow, cLow = np.linalg.lstsq(A, angVelLow)[0]
print mLow, cLow
plt.plot(cmdLow, angVelLow, '*')
plt.plot(cmdLow, mLow*cmdLow + cLow)

In [ ]:
ax1=plt.subplot(2, 1, 1)

ax1.plot(mot0_cmdTime, mot0.cmd, 'b')
ax1.plot(mot0_fbTime, np.clip(invfit(mot0.angVel.values, 100, 300, 600, 6*12.5/11.1, 3.5*12.5/11.1, 2.14*12.5/11.1), 0, None), 'go',
         linestyle='None', markersize=1)

ax2=plt.subplot(2, 1, 2)
ax2.plot(mot0_cmdTime, mot0.cmdVel - mot0.angVel)

In [ ]:
cmd = mot0.cmd.values.astype("float64")
angVel = mot0.angVel.values
plt.plot(cmd, angVel, '*')
plt.plot(cmd, fit(cmd, 100, 300, 600, 6*12.5/11.1, 3.5*12.5/11.1, 2.14*12.5/11.1), 'go', linestyle='None')
plt.plot(cmdLow, mLow*cmdLow + cLow)
plt.plot(cmdMed, mMed*cmdMed + cMed)
plt.plot(cmdHi, mHi*cmdHi + cHi)

In [ ]:
fig, ax1 = plt.subplots()
torque_cut = torque[50:-35]

ax1.plot(torque_cut['Time (s)'], torque_cut['Torque (N\xc2\xb7m)'], '*')
mot0_fbTime_zero = mot0_fbTime - 1261.1

ax2 = ax1.twinx()
ax2.plot(mot0_fbTime_zero, mot0.angVel)

idxMax = mot0.angVel.idxmax()
t0 = mot0_fbTime_zero[0]
tMax = mot0_fbTime_zero[idxMax]
tLast = mot0_fbTime_zero.iloc[-1]
torque_funclist = [lambda x: (x - t0) * mot0.angVel.max() / (tMax - t0),
                   lambda x: mot0.angVel.max() + (x - tMax) * mot0.angVel.max() / (tMax - tLast)]

angVel_torque_mapped = np.piecewise(torque_cut['Time (s)'].values,
                                    [torque_cut['Time (s)'].values < tMax, torque_cut['Time (s)'].values >=tMax],
                                    torque_funclist)
ax2.plot(torque_cut['Time (s)'], angVel_torque_mapped)
torque_fit = np.polyfit(angVel_torque_mapped, torque_cut['Torque (N\xc2\xb7m)'], 2)
torque_poly = np.poly1d(torque_fit)
ax1.plot(torque_cut['Time (s)'], torque_poly(angVel_torque_mapped), '*')

align_yaxis(ax1, 0, ax2, 0)
torque_poly

In [ ]:
fig, ax1 = plt.subplots()
thrust_cut = thrust[50:-35]

ax1.plot(thrust_cut['Time (s)'], thrust_cut['Thrust (kgf)'], '*')
mot0_fbTime_zero = mot0_fbTime - 1261.1

ax2 = ax1.twinx()
ax2.plot(mot0_fbTime_zero, mot0.angVel)

idxMax = mot0.angVel.idxmax()
t0 = mot0_fbTime_zero[0]
tMax = mot0_fbTime_zero[idxMax]
tLast = mot0_fbTime_zero.iloc[-1]
thrust_funclist = [lambda x: (x - t0) * mot0.angVel.max() / (tMax - t0),
                   lambda x: mot0.angVel.max() + (x - tMax) * mot0.angVel.max() / (tMax - tLast)]

angVel_thrust_mapped = np.piecewise(thrust_cut['Time (s)'].values,
                                    [thrust_cut['Time (s)'].values < tMax, thrust_cut['Time (s)'].values >=tMax],
                                    thrust_funclist)
ax2.plot(thrust_cut['Time (s)'], angVel_thrust_mapped)

thrust_fit = np.polyfit(angVel_thrust_mapped, thrust_cut['Thrust (kgf)'], 2)
thrust_poly = np.poly1d(thrust_fit)
ax1.plot(thrust_cut['Time (s)'], thrust_poly(angVel_thrust_mapped), '*')

align_yaxis(ax1, 0, ax2, 0)
thrust_poly